In [2175]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2176]:
import numpy as np

import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [2177]:
def get_p_values_old(v,pi,j):
    '''
    This function calculates the p values for a given
    v values set and a polytropic process with index j.
    
    Inputs:
    v: 1d array of float values for volume
    pi: float value for inital state pressure
    j: float value for polytropic index
    
    Returns:
    p: 1d array of float pressure values
    '''
    
    p = pi*(v[0]/v)**j
    return p

In [2178]:
def calculate_work_old(v,p):
    '''
    This function calculates the total Work done in the process
    described by the point arrays (v,p). This calculation
    is done by applying numerical integration based on:
    
    dW = -pdV
    
    The fact that all values in v are equidistant is exploited to
    simplify the calculation.
    
    Inputs:
    v: 1d array of float volume values for the process (in L)
    p: 1d array of float pressure values for the process (in atm)
    
    Returns:
    W: float value for the work done by the system (negative) or on the system (positive) (in J)
    
    '''
    W= -sum(p)*(v[pts-1] - v[0])/pts
    return W

In [2179]:
def calculate_energy_old(vi,pi,vf,pf):
    '''
    This function calculates the total internal-energy change in any process
    that goes from (vi,pi) to (vf,pf). This calculation
    is done exploiting the fact that internal-energy is a function of state.
    
    dU = 3/2*N*R*dT
    
    Inputs:
    vi: float value for initial volume (in L)
    pi: float value for initial pressure (in atm)
    vf: float value for final volume (in L)
    pf: float value for final pressure (in atm)  
    
    Returns:
    dU: float value for the internal-energy change of the system (in J)
    '''
    dU = 1.5*(vf*pf - vi*pi)
    return dU

In [2180]:
def update_figure_old(change):
    '''
    This function recalculates parameters and replots the figure
    whenever a widget value changes.
    '''
    
    vi = vi_slider.value
    pi = pi_slider.value
    vf = vf_slider.value
    j = j_slider.value
    
        
    v = np.linspace(vi,vf,pts)
    p = get_p_values(v,pi,j)
    pf = p[pts-1]
    pf_widget.value = '%.2f' % pf
    
    Ti = vi*pi/N/R
    Tf = vf*pf/N/R
    Ti_widget.value = '%.2f' % Ti
    Tf_widget.value = '%.2f' % Tf
    
    work = calculate_work(v,p)
    energy = calculate_energy(vi,pi,vf,pf)
    
    work_value_text.value = '%.2f' % work
    energy_value_text.value = '%.2f' % energy
    
    heat_value_text.value = '%.2f' % (energy - work)
    
    curve.x = v
    curve.y = p
    InitialPoint.x = [vi]
    InitialPoint.y = [pi]
    FinalPoint.x = [vf]
    FinalPoint.y = [pf]

In [2181]:
def f(cahnge):
    return None

In [2182]:
def get_isochore(vi,pi):
    v_values = np.full((pts), vi)
    p_values = np.linspace(p_min,p_max,pts)
    return v_values, p_values

In [2183]:
def get_isobar(vi, pi):
    v_values = np.linspace(v_min, v_max, pts)
    p_values = np.full((pts), pi)
    return v_values, p_values

In [2184]:
def get_isotherm(vi, pi):
    v_values = np.linspace(v_min, v_max, pts)
    p_values = np.empty((pts))
    for i in range(pts):
        p_values[i] = pi* vi/v_values[i]
    return v_values, p_values

In [2185]:
def get_isenthropic(vi, pi, gamma):
    v_values = np.linspace(v_min, v_max, pts)
    p_values = np.empty((pts))
    for i in range(pts):
        p_values[i] = pi* (vi/v_values[i])**gamma
    return v_values, p_values

In [2186]:
def get_pf(vi, pi, vf,gamma, process):
    if process == 0:
        pf = 10.0
    elif process == 1:
        pf = pi
    elif process == 2:
        pf = pi* vi/vf
    elif process == 3:
        pf = pi* (vi/vf)**gamma
    return pf

In [2187]:
def change_initial_point(change):
    return None

In [2188]:
def change_final_volume(change):
    return None

In [2189]:
def update_labels(Ti,pf,Tf):
    Ti_text.value = '%.2f' % Ti
    pf_text.value ='%.2f' % pf
    Tf_text.value ='%.2f' % Tf

In [2190]:
def change_mol(change):
    vi = vi_slider.value
    pi = pi_slider.value
    vf = vf_slider.value
    N = mol_slider.value
    
    Ti = vi*pi/N/R
    pf = pi* vi/vf
    Tf = vf*pf/N/R
    update_labels(Ti,pf,Tf)

In [2191]:
def get_marks(vi, pi, gamma):
    
    
    x_values = []
    y_values = []
    
    isochore = get_isochore(vi, pi)
    x_values.append(isochore[0])
    y_values.append(isochore[1])
    
    isobar = get_isobar(vi, pi)
    x_values.append(isobar[0])
    y_values.append(isobar[1])
    
    isotherm = get_isotherm(vi, pi)
    x_values.append(isotherm[0])
    y_values.append(isotherm[1])
    
    isenthropic = get_isenthropic(vi, pi, gamma)
    x_values.append(isenthropic[0])
    y_values.append(isenthropic[1])
    
    return x_values, y_values

In [2192]:
def update_points(change):
    vi = vi_slider.value
    pi = pi_slider.value
    vf = vf_slider.value
    gamma = gamma_dropdown.value
    process = process_dropdown.value
    N = mol_slider.value

    Ti = vi*pi/N/R #This will only cahnge
    pf = get_pf(vi, pi, vf, gamma, process)
    Tf = vf*pf/N/R
    
    # Update the marks
    InitialPoint.x = [vi]
    InitialPoint.y = [pi]
    FinalPoint.x = [vf]
    FinalPoint.y = [pf]
    
    update_labels(Ti,pf,Tf)

In [2193]:
def change_process(change):
    
    vi = vi_slider.value
    pi = pi_slider.value
    vf = vf_slider.value
    gamma = gamma_dropdown.value
    process = process_dropdown.value
    
    # Update opacities
    op = [0.3,0.3,0.3,0.3]
    op[process] = 1.0
    isocurves.opacities = op
    
    # Update final point tracer
    update_points(None)

In [2194]:
def update_figure(change):
    
    vi = vi_slider.value
    pi = pi_slider.value
    gamma = gamma_dropdown.value
    
    # Update the isocurves
    x_values, y_values = get_marks(vi, pi, gamma)
    isocurves.x = x_values
    isocurves.y = y_values
    
    update_points(None)

In [2195]:
# Global Parameters
R = 0.082057 # In atm*L/mol/K

pts = 200 # Number of points for each process

# Limits of parameters (volumes in L, pressures in atm)
v_min = 0.01
v_max = 20.0
p_min = 0.01
p_max = 20.0

# Default values
vi = 1.0
pi = 14.0
vf = 14.0
gamma = 5.0/3.0 # Adiabatic index (5/3 for monoatomic gases, 7/5 for diatomic gases at room temperature)
process = 2 # Index for each process: {0: 'isochoric', 1: 'isobaric', 2: 'isothermal', 3: 'isenthropic' or 'adiabatic'}
N = 1.0 # Number of mols of gas


# Default final state
Ti = vi*pi/N/R
vf = 14.0
pf = 1.0

# Get curves
isocurves_opacities = [0.3,0.3,1.0,0.3]


# Final states parameters (once p have been calculated)
Tf = vf*pf/N/R
Tf_text = '%.2f' % Tf

# Energies
W = 0.0
DU = 0.0
Q = 0.0



########################
###CREATE THE FIGURES###
########################

fig_122_001 = bq.Figure(title='Prozesu sinpleak',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                toolbar = True,
                layout=widgets.Layout(width='100%')
    )


scale_x = bqs.LinearScale(min = v_min, max = v_max)
scale_y = bqs.LinearScale(min = p_min, max = p_max)

axis_x = bqa.Axis(scale=scale_x,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(v_min, v_max, 11),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(
                scale=scale_y,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values= np.linspace(p_min, p_max, 6),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_122_001.axes = [axis_x,axis_y]


########################
####CREATE THE MARKS####
########################

marks = []

(x_values, y_values) = get_marks(vi, pi, gamma)

colors = ['#0079c4','#f09205','#21c400','#dd4e4f']

isocurves = bqm.Lines(
                x = x_values, 
                y = y_values, 
                scales = {'x': scale_x, 'y': scale_y},
                colors = colors,
                opacities = isocurves_opacities,
                #fill_colors = ['green'],
                #fill_opacities = [0.0],
                #fill = 'bottom',
                labels = ['Isokoroa', 'Isobaroa', 'Isotermoa', 'Adiabatikoa'],
                display_legend=True
)

marks.append(isocurves)

InitialPoint = bqm.Scatter(
    name = 'Initial Point',
    x = [vi],
    y = [pi],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['orange'],
    names = [],
    labels=['Hasierako egoera'],
    display_legend = True
)      

marks.append(InitialPoint)

FinalPoint = bqm.Scatter(
    name = 'Final Point',
    x = [vf],
    y = [pf],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['red'],
    names = [],
    labels=['Bukaerako egoera'],
    display_legend = True
)      

marks.append(FinalPoint)

fig_122_001.marks = marks

########################
######  WIDGETS  #######
########################


## Left block (Initial state and process block)

gamma_dropdown = widgets.Dropdown(
    options=[('Monoatomikoa',5.0/3.0), ('Diatomikoa',7.0/5.0)],
    value=gamma,
    description='Gasa',
    disabled=False,
    layout=widgets.Layout(width='100%')
)

gamma_dropdown.observe(update_figure, 'value')

mol_slider = widgets.FloatSlider(
    value=1.0,
    min=v_min,
    max=v_max,
    step=0.1,
    description='$N$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

mol_slider.observe(change_mol, 'value')



vi_slider = widgets.FloatSlider(
    value=vi,
    min=v_min,
    max=v_max,
    step=0.1,
    description='$v_i$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

vi_slider.observe(update_figure, 'value')


pi_slider = widgets.FloatSlider(
    value=pi,
    min=p_min,
    max=p_max,
    step=0.1,
    description='$p_i$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

pi_slider.observe(update_figure, 'value')

Ti_text = widgets.Label(value='%.2f' % Ti)


process_dropdown = widgets.Dropdown(
    options=[('Isobaroa',1), ('Isotermoa',2), ('Adiabatikoa',3)],
    value=2,
    description='Prozesua',
    disabled=False,
    layout=widgets.Layout(width='100%')
)

process_dropdown.observe(change_process, 'value')


## Center block (figure block)

vf_slider = widgets.FloatSlider(
    value=vf,
    min=v_min,
    max=v_max,
    step=0.1,
    description='$v_f$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

vf_slider.observe(update_points, 'value')

pf_text = widgets.Label(value='%.2f' % pf)
Tf_text = widgets.Label(value='%.2f' % Tf)


## Right block (energy block)

work_value_text = widgets.Label(value='$W$: ' + '%.2f' % W + ' $J$')
energy_value_text = widgets.Label(value='$\Delta U$: ' + '%.2f' % DU + '$J$' )
heat_value_text = widgets.Label(value='$Q$: ' + '%.2f' % Q + '$J$')

show_work_check = widgets.Checkbox(
    description='Erakutsi lana',
    disabled=False,
    value=False,
)

show_work_check.observe(f,'value')


########################
######  LAYOUT  ########
########################


## Left Block ##
left_block_122_000 = widgets.VBox([], layout=widgets.Layout(width='20%', align_items='center'))
left_block_122_000.children = [gamma_dropdown, mol_slider, widgets.Label(value="Hasierako egoera:"), vi_slider, pi_slider,
                               widgets.HBox([widgets.Label(value='$T_i=$'),Ti_text,widgets.Label(value='$K$')]),
                               process_dropdown]
                              
## Center Block ##
center_block_122_000 = widgets.VBox([fig_122_001], layout=widgets.Layout(width='60%', align_items='center'))
center_block_122_000.children = [fig_122_001,vf_slider,
                                 widgets.HBox([widgets.Label(value='$p_f=$'),pf_text,widgets.Label(value='$atm$')]),
                                 widgets.HBox([widgets.Label(value='$T_f=$'),Tf_text,widgets.Label(value='$K$')])
                                ]

## Right Block ##
right_block_122_000 = widgets.VBox([], layout=widgets.Layout(width='20%'))
right_block_122_000.children = [work_value_text, energy_value_text, heat_value_text, show_work_check]
                               
                                

## Main Block ##

main_block_122_000 = widgets.HBox([],layout=widgets.Layout(width='100%', align_items='center'))
main_block_122_000.children = [left_block_122_000, center_block_122_000,right_block_122_000]

main_block_122_000